### 前言

决策树的优缺点如下:
* 优点：计算复杂度不高，输出结果易于理解，对中间值的缺少不敏感，可以处理不相关特征数据
* 缺点：可能会产生过度匹配问题

本次将实现一个简易的ID3决策树算法。ID3决策树依据信息增益选择最优划分属性。

下面实现的决策树只能处理离散数据。

### ID3决策树从零实现

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#计算信息熵
#信息熵越小，代表划分的数据集纯度越高
def calcEnt(dataset):
    n,m=dataset.shape
    labelCounts={}
    for i in range(n):
        currentLabel=dataset[i][-1]#取最后一列键值
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel]=0
        labelCounts[currentLabel]+=1
    Ent=0.0
    for key in labelCounts:
        prob=float(labelCounts[key])/n
        Ent-=prob*np.log2(prob)#计算信息熵
    return Ent

In [3]:
#测试计算信息熵
#mydata中每一行的最后一列代表类别
mydata=np.array([[1,1,1],
                [1,1,1],
                [1,0,0],
                [0,1,0],
                [0,1,0]])
print(calcEnt(mydata))

0.9709505944546686


In [4]:
#划分数据集
def splitDataset(dataset,axis,value):#带划分的数据集 数据集特征 需要返回的特征值
    dataset=dataset[dataset[:,axis]==value]
    return np.delete(dataset,axis,axis=1) #返回包含当前属性的数据集

In [5]:
#测试划分数据
print(splitDataset(mydata,0,1))

[[1 1]
 [1 1]
 [0 0]]


In [6]:
#选择最优的属性划分
def chooseBestFeature(dataset):  #dataset每一行的最后一个元素是类别标签
    numFeatures=dataset.shape[1]-1  #获取特征数量
    bestEnt=calcEnt(dataset)
    bestGain=0.0
    bestFeature=-1
    for i in range(numFeatures):
        featData=dataset[:,i]
        uniqueVals=np.unique(featData)  #获取当前属性的所有取值
        newEnt=0.0
        for value in uniqueVals:  #遍历所有可能的取值
            subDataset=splitDataset(dataset,i,value)
            prob=subDataset.shape[0]/float(dataset.shape[0])
            newEnt+=prob*calcEnt(subDataset)
        infoGain=bestEnt-newEnt #计算信息增益
        if(infoGain>bestGain):
            bestGain=infoGain
            bestFeature=i
    return bestFeature

In [7]:
#测试划分最优属性
mydata
print(chooseBestFeature(mydata))

0


In [8]:
#递归创建决策树
#递归的终止条件:
#      1.当前样本的类别标签完全相同，此时直接返回该类标签
#      2.使用完了所有特征，仍然不能将数据划分成仅包含位移类别的分组
#         此时挑选出现次数最多的类别作为返回值
def majorityCnt(classList):#classList的shape为(n,1)
    return np.argmax(np.bincount(np.squeeze(classList)))
    
def createTree(dataset,labels):#dataset每一行的最优一列元素是类别标签，labels是属性列表
    classList=dataset[:,-1]
    if classList[classList==classList[0]].shape[0]==classList.shape[0]:
        return classList[0]  #类别完全相同则停止继续划分
    if dataset.shape[1]==1:
        return majorityCnt(classList) #遍历完所有特征时返回出现次数最多的
    #创建树
    bestFeat=chooseBestFeature(dataset)
    bestFeatLabel=labels[bestFeat]
    myTree={bestFeatLabel:{}}  #通过字典构造数
    subLabels=np.delete(labels,bestFeat)  #删除当前属性
    uniqueVals=np.unique(dataset[:,bestFeat]) #获取属性的所有可能取值
    for value in uniqueVals:
        myTree[bestFeatLabel][value]=createTree(splitDataset(dataset,bestFeat,value),subLabels)
    
    return myTree

In [9]:
#测试构建决策树
labels=np.array(['no surfacing','flippers'])
print(mydata)
print(labels)
myTree=createTree(mydata,labels)
print(myTree)

[[1 1 1]
 [1 1 1]
 [1 0 0]
 [0 1 0]
 [0 1 0]]
['no surfacing' 'flippers']
{'no surfacing': {0: 0, 1: {'flippers': {0: 0, 1: 1}}}}


In [10]:
#使用决策树进行分类
def classifier(inputTree,featLabels,testVec):
    firstStr=list(inputTree.keys())[0]
    secondDict=inputTree[firstStr]   #决策树下一层
    featIndex=np.where(featLabels==firstStr)[0][0]
    for key in secondDict.keys():
        if testVec[featIndex]==key:  #该特征值等于当前key,往下走
            if type(secondDict[key]).__name__=='dict':  #若为树结构
                classLabel=classifier(secondDict[key],featLabels,testVec)
            else:
                classLabel=secondDict[key]  #为叶子节点
    return classLabel


In [11]:
#测试用决策树分类
if classifier(myTree,labels,[1,1])== 1:
    print("yes")
else:
    print("no")

yes
